# Crop prediction Model

Imports neccessary 

In [1]:
import numpy as np
import pandas as pd 
import tensorflow as tf 
import matplotlib.pyplot as plt
from sklearn import preprocessing 
from sklearn.preprocessing import StandardScaler
import seaborn as  sb 
sb.set()

Dataset exploration

In [2]:
crop_data = pd.read_csv('crop_dataset.csv')
crop_data.head()

,Crop,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,Cassava,150,50,100,5.5,1.5,60
1,Cassava,150,50,100,5.5,1.5,62
2,Cassava,150,50,100,5.5,1.5,64
3,Cassava,150,50,100,5.5,1.5,66
4,Cassava,150,50,100,5.5,1.5,68


In [3]:
crops = crop_data['Crop'].unique()
crops

array(['Cassava', 'Vanilla', 'Coffee', 'Cotton ', 'Tea', 'Tobacco',
       'Groundnuts', 'Yams', 'Maize (corn)', 'Beans', 'Irish Potato',
       'Matooke', 'Sweet Banana', 'Sugarcane'], dtype=object)

In [4]:
crop_data.shape

(37581648, 7)

In [5]:
crop_data.dropna()

,Crop,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,Cassava,150,50,100,5.5,1.5,60
1,Cassava,150,50,100,5.5,1.5,62
2,Cassava,150,50,100,5.5,1.5,64
3,Cassava,150,50,100,5.5,1.5,66
4,Cassava,150,50,100,5.5,1.5,68
...,...,...,...,...,...,...,...
37581643,Sugarcane,200,80,180,6.5,2.5,72
37581644,Sugarcane,200,80,180,6.5,2.5,74
37581645,Sugarcane,200,80,180,6.5,2.5,76
37581646,Sugarcane,200,80,180,6.5,2.5,78


In [6]:
from sklearn.preprocessing import LabelEncoder

label_encoder = LabelEncoder()
target = label_encoder.fit_transform(crop_data.iloc[:,:1])
target

H:\Anaconda\lib\site-packages\sklearn\preprocessing\_label.py:115: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


array([1, 1, 1, ..., 8, 8, 8])

In [7]:
crop_data.shape

(37581648, 7)

In [8]:
un_scaled_inputs = crop_data.iloc[:,1:]
un_scaled_inputs

,N (mg/kg),P (mg/kg),K (mg/kg),pH,EC(uS/cm),MOISTURE (%)
0,150,50,100,5.5,1.5,60
1,150,50,100,5.5,1.5,62
2,150,50,100,5.5,1.5,64
3,150,50,100,5.5,1.5,66
4,150,50,100,5.5,1.5,68
...,...,...,...,...,...,...
37581643,200,80,180,6.5,2.5,72
37581644,200,80,180,6.5,2.5,74
37581645,200,80,180,6.5,2.5,76
37581646,200,80,180,6.5,2.5,78


In [9]:
# target = crop_data.iloc[:,0]
# target

scaling inputs 

In [10]:
scaler = StandardScaler()
scaled_inputs = scaler.fit_transform(un_scaled_inputs)
scaled_inputs

array([[-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.36091184],
       [-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.24596734],
       [-0.17131778, -0.6124411 , -1.63087258, -1.0566009 , -0.67650084,
        -0.13102284],
       ...,
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.55864417],
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.67358867],
       [ 1.96016098,  1.47317381,  2.56785036,  0.84500193,  1.55786254,
         0.78853317]])

In [11]:
shuffled_indices = np.arange(scaled_inputs.shape[0])
np.random.shuffle(shuffled_indices)

shuffled_inputs = scaled_inputs[shuffled_indices]
shuffled_targets = target[shuffled_indices]
shuffled_targets

array([ 7,  5, 10, ...,  7,  5, 12])

In [12]:
sample_count = shuffled_inputs.shape[0]

training_count, validation_count,test_count = int(0.8*sample_count),int(0.1*sample_count),int(0.1*sample_count)
print(training_count,validation_count,test_count)

30065318 3758164 3758164


spliting the data

In [13]:
train_inputs,train_targets = shuffled_inputs[:training_count],shuffled_targets[:training_count]
validation_input,validation_targets = shuffled_inputs[training_count:training_count+validation_count],shuffled_targets[training_count:training_count+validation_count]
test_inputs,test_targets = shuffled_inputs[training_count+validation_count:],shuffled_targets[training_count+validation_count:]

In [14]:
np.savez('Train_data',inputs =train_inputs,targets = train_targets)
np.savez('validation_data',inputs =validation_input,targets = validation_targets)
np.savez('test_data',inputs =test_inputs,targets = test_targets)

loading the saved data

In [15]:
trainData = np.load('Train_data.npz',allow_pickle=True)
train_inputs,train_targets = trainData['inputs'].astype(np.float64),trainData['targets']
validation_data = np.load('validation_data.npz',allow_pickle=True)
validation_inputs,validation_targets = validation_data['inputs'].astype(np.float64),validation_data['targets']
test_data = np.load('test_data.npz',allow_pickle=True)
test_inputs,test_targets = test_data['inputs'].astype(np.float64),test_data['targets']

In [17]:
print(train_targets)
original_classes = label_encoder.inverse_transform(train_targets)
print(original_classes)

[ 7  5 10 ...  1  5  3]
['Matooke' 'Irish Potato' 'Tea' ... 'Cassava' 'Irish Potato' 'Cotton ']


## MODEL 

In [83]:
input_size = 6 
output_size = 14
hidden_layer_size = 100

model = tf.keras.Sequential([
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'relu'),
                             tf.keras.layers.Dense(hidden_layer_size, activation = 'softmax'),
                            ])

model.compile(optimizer = 'adam',loss = 'sparse_categorical_crossentropy', metrics = ['accuracy'])

batch_size = 100
num_epoches = 100
early_stp = tf.keras.callbacks.EarlyStopping(patience =1)

model.fit(train_inputs,
         train_targets,
         batch_size = batch_size,
         epochs = num_epoches,
          callbacks = [early_stp],
         validation_data = (validation_inputs,validation_targets),
         verbose = 1
         )

Epoch 1/100
300654/300654 [==============================] - 1585s 5ms/step - loss: 0.2975 - accuracy: 0.8086 - val_loss: 0.2840 - val_accuracy: 0.8127
Epoch 2/100
300654/300654 [==============================] - 1559s 5ms/step - loss: 0.2865 - accuracy: 0.8123 - val_loss: 0.2845 - val_accuracy: 0.8132


In [96]:
test_loss, test_accuracy = model.evaluate(test_inputs,test_targets)

In [86]:
test_data = test_inputs[-1:,:]
predictionns = model.predict(test_data)

1/1 [==============================] - 0s 291ms/step


In [87]:
print(test_data)

[[ 0.68127372 -0.75148209 -1.00106414 -0.67628034 -0.89993718  0.78853317]]


In [88]:
predictionns

array([[4.6537340e-30, 3.2309939e-16, 1.4212319e-04, 2.7417895e-35,
        1.4281404e-14, 7.0095389e-06, 7.1124875e-24, 7.2324047e-17,
        5.3601680e-18, 5.4361135e-17, 2.4302528e-06, 1.0457059e-31,
        4.7603813e-01, 5.2381027e-01, 5.5783639e-21, 5.2168920e-21,
        4.8162854e-21, 4.9331153e-21, 6.6303361e-21, 7.1007324e-21,
        4.3288009e-21, 4.9745568e-21, 8.4701445e-21, 6.9606037e-21,
        6.9104155e-21, 5.9974633e-21, 6.0965590e-21, 8.8629325e-21,
        4.3601235e-21, 5.3024632e-21, 5.5044007e-21, 8.9371485e-21,
        5.6323334e-21, 4.9054549e-21, 6.2658249e-21, 4.3290655e-21,
        9.2876001e-21, 6.4878724e-21, 7.0001993e-21, 7.3511614e-21,
        4.7896092e-21, 5.4507002e-21, 5.9486120e-21, 6.0895167e-21,
        4.6990487e-21, 4.4993867e-21, 6.4164457e-21, 6.9167179e-21,
        6.6963986e-21, 5.4498690e-21, 7.7903636e-21, 4.1156133e-21,
        5.9048191e-21, 8.8217471e-21, 5.2334558e-21, 7.8896236e-21,
        4.2148677e-21, 7.8237481e-21, 6.8374144e

In [89]:
predicted_class = np.argmax(predictionns)
predicted_crop = label_encoder.inverse_transform([predicted_class])
print(predicted_crop)

['Yams']


In [90]:
test_targets

array(['Sweet Banana', 'Groundnuts', 'Matooke', ..., 'Matooke',
       'Irish Potato', 'Vanilla'], dtype=object)

In [97]:
determinants = ['N','P','K','pH','EC','moisture']
det_values = [[],]

for determinant in determinants:
        value = float(input(f'Please enter the value of {determinant}'))
        det_values[0].append(value)

scaled_query = scaler.transform(det_values)
print(scaled_query)
print(det_values)

Please enter the value of N 100
Please enter the value of P 150
Please enter the value of K 200
Please enter the value of pH 5
Please enter the value of EC 3
Please enter the value of moisture 60


H:\Anaconda\lib\site-packages\sklearn\base.py:450: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


[[-2.30279653  6.33960861  3.6175311  -2.00740232  2.67504424 -0.36091184]]
[[100.0, 150.0, 200.0, 5.0, 3.0, 60.0]]


In [98]:
prediction = model.predict(scaled_query)
predicted_class = np.argmax(prediction)
predicted_crop = label_encoder.inverse_transform([predicted_class])
print(predicted_crop)

1/1 [==============================] - 2s 2s/step
['Maize (corn)']
